In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import yfinance as yf
import pandas as pd
import datetime as dt
import requests
from bs4 import BeautifulSoup
import nltk
from tqdm import tqdm

from finBERT_functions import *

In [ ]:
ticker_news = yf.Ticker("GOOG").news

dict_list = []

labels = {0:'neutral', 1:'positive', 2:'negative'}

for news in tqdm(ticker_news):
    timestamp = news['providerPublishTime']
    datetime = dt.datetime.fromtimestamp(timestamp)

    url = news['link']
    response = requests.get(url)

    if response.status_code != 200:
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    

    article = soup.find('div', class_='caas-body').text

    # tokenize the article as sentences
    sentences = nltk.sent_tokenize(article)

    news_dict = {
        'date': datetime,
        'title': news['title'],
        'link': news['link'],
    }

    news_dict.update(get_sentiment(sentences))

    dict_list.append(news_dict)

    print(news_dict)

df = pd.DataFrame(dict_list, index=[x['date'] for x in dict_list]).drop(columns=['date'])

display(df)